# BART sequence framework

### 27th annual meeting of DACH-ISMRM + Nordic chapter

- September 14th 2025


The purpose of this notebook is to introduce basic concepts of the [BART toolbox](https://mrirecon.github.io/bart), especially the usage of the seq - tool, export to a pulseq sequence and actual scanning.

We recommend checking other points of reference, primarily available in the following locations:
- [BART Workshops](https://github.com/mrirecon/bart-workshop)
- [BART Webinars](https://github.com/mrirecon/bart-webinars) (and [webinar recordings](https://www.youtube.com/@bart-toolboxdepartmental7435/playlists)\)


**Author**: [Daniel Mackner](mailto:daniel.mackner@tugraz.at)

**Institution**: Graz University of Technology

Based on previous material written by [Moritz Blumenthal](mailto:blumenthal@@tugraz.at), [H. Christian M. Holme](mailto:Holme@tugraz.at) and [Jon Tamir](mailto:jtamir@utexas.edu)

## Content


## Outline
1. Bart Setup
1. Sequence concepts
1. pulse - tool
1. seq - tool
1. pulseq export

* BART sequence
  * config
  * events and pulses
  * gradients / moments
  * trajectories
  * (FoV shift)
  * pulseq - export


## Installation

You can find more information on BART on the [Website](https://mrirecon.github.io/bart/).

## 1 Setup BART
This notebook is designed to run on a local system. It uses the python kernel, however, almost all commands use the `%%bash` cell magic to be executed in a `bash` subshell.

### 1.1 Local Usage
- Install bart from its [github repository](https://github.com/mrirecon/bart)
- Set the `BART_TOOLBOX_PATH` to the BART directory and add it to the `PATH`

```bash
export BART_TOOLBOX_PATH=/path/to/bart  
export PATH=$BART_TOOLBOX_PATH:$PATH
```

Although the simplest way to call the BART CLI tools is from a terminal, there are also wrapper functions that allow the tools to be used from Matlab and Python. These are located under the `$BART_TOOLBOX_PATH/matlab` and `$BART_TOOLBOX_PATH/python` directories.

In [ ]:
import os, sys

os.environ['COLAB'] = 'true' if ('google.colab' in sys.modules) else 'false'

In [ ]:
%%bash

# MyBinder has BART already installed via the container
if $COLAB; then

  # Install BARTs dependencies
  apt-get install -y make gcc-12 g++-12 libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

  # Clone Bart
  [ -d /content/bart ] && rm -r /content/bart
  git clone https://github.com/mrirecon/bart/ bart &> /dev/null

fi

In [ ]:
%%bash

if $COLAB; then

  cd bart
  git checkout d2896b04cb9a0b69c4ec26bbf790d73fcce29638

  # Configuration
  COMPILE_SPECS=" PARALLEL=1
                  OPENBLAS=1
                  BLAS_THREADSAFE=1
                  CC=gcc-12"

  printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

  # Compile BART
  make &> /dev/null && echo ok
fi

In [ ]:
if os.environ['COLAB'] == 'true':
    os.environ['BART_TOOLBOX_PATH'] = "/content/bart"
    os.environ['OMP_NUM_THREADS']="4"

# Add the BARTs toolbox to the PATH variable
os.environ['PATH'] = os.environ['BART_TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['BART_TOOLBOX_PATH'] + "/python")

In [ ]:
%%bash

echo "# The BART used in this notebook:"
which bart
echo "# BART version: "
bart version

In [ ]:
# get a list of BART commands by running bart with no arguments:
! bart

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.environ['BART_TOOLBOX_PATH'] + "/python")

from bart import cfl

def plot_from_cfl(name):
    """Plot from a BART cfl"""

    if not os.path.exists(name + '.cfl'):
        FileNotFoundError(f"{name + '.cfl'} does not exist")
    if not os.path.exists(name + '.hdr'):
        FileNotFoundError(f"{name + '.hdr'} does not exist")

    data = np.array(cfl.readcfl('./' + name))

    try:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))
        ax.plot(np.real(data))
        ax.grid()
    except:
        print("Could not plot data, probably not 1D.")
        plt.close(fig)
    
def plot_mom_from_cfl(name, ylim=-1):
    """Plot three axes from a BART cfl"""

    if not os.path.exists(name + '.cfl'):
        FileNotFoundError(f"{name + '.cfl'} does not exist")
    if not os.path.exists(name + '.hdr'):
        FileNotFoundError(f"{name + '.hdr'} does not exist")

    data = np.array(cfl.readcfl('./' + name))

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 3))

    ax[0].plot(np.real(data[0,:]))
    ax[1].plot(np.real(data[1,:]))
    ax[2].plot(np.real(data[2,:]))

    titles = ['RO', 'PE', 'SL']
    for i in range(3):
        ax[i].grid()
        if ylim > 0:
                ax[i].set_ylim(-ylim,ylim)
        ax[i].set_title(titles[i])
    

def plot_grad_from_cfl(name):
    """Plot three axes from a BART cfl"""

    plot_mom_from_cfl(name, 25e-3)
    

def bart_show(name, title=None, vmin=None, vmax=None, cmap='gray', cbar_label='', mag=True, fsize=10):

    if not os.path.exists(name + '.cfl'):
        FileNotFoundError(f"{name + '.cfl'} does not exist")
    if not os.path.exists(name + '.hdr'):
        FileNotFoundError(f"{name + '.hdr'} does not exist")

    # Import data
    data = np.array(cfl.readcfl('./' + name))
    data = np.abs(data) if mag else data

    fig = plt.figure(figsize=(fsize,fsize))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(data, interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
    if title:
      plt.title(title)

    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.xaxis.set_ticks_position('none')
    ax1.yaxis.set_ticks_position('none')
    ax1.set_axis_off()


    plt.show()


## From pulse to pulseq

### RF pulses

We can generate pulses with the *pulse* tool

In [ ]:
%%bash

bart pulse -h

In [ ]:
%%bash

bart pulse --sinc pulse

bart cabs pulse pulse_cabs
bart carg pulse pulse_carg

In [ ]:
plot_from_cfl('pulse_cabs')
plot_from_cfl('pulse_carg')

In [ ]:
%%bash

bart pulse --hypsec pulse

bart cabs pulse pulse_cabs
bart carg pulse pulse_carg

In [ ]:
plot_from_cfl('pulse_cabs')
plot_from_cfl('pulse_carg')

## Sequence block

The computation of a single sequence block can be accessed with the *seq* tool.

If we do not modify the sequence parameters via the interface, a default sequence configuration is used.

Parameters can be found in *bart*: src/seq/config.c (or expand this cell)


```c
const struct seq_config seq_config_defaults = {

	.phys = {
		.tr = 3.11E-3,
		.te = 1.90E-3,
		.dwell = 4.E-6,
		.os = 2.,
		.contrast = CONTRAST_RF_RANDOM,
		.rf_duration = 620.E-6,
		.flip_angle = 6.,
		.bwtp = 3.8,
	},

	.geom = {
		.fov = .256,
		.slice_thickness = .006,
		.shift = { [0 ... MAX_SLICES - 1] = { 0., 0., 0. } },
		.baseres = 256,
		.mb_factor = 1,
		.sms_distance = .020,
	},

	.enc = {
		.pe_mode = PEMODE_RAGA,
		.tiny = 1,
		.aligned_flags = 0,
	},

	.magn = {
		.mag_prep = PREP_OFF,
		.ti = 0.,
		.init_delay = 0.,
		.inv_delay_time = 0.,
	},

	.sys = {
		.gamma = 42.575575E6,
		.b0 = 2.893620,
		.grad.inv_slew_rate = .007848885540911,
		.grad.max_amplitude = .024,
		.coil_control_lead = 100.E-6,
		.min_duration_ro_rf = 213.E-6,
		.raster_grad = 1.E-5,
		.raster_rf = 1.E-6,
		.raster_dwell = 1.E-7,
	},

	.order = { [0 ... DIMS - 1] = 1 },
	.loop_dims = { [0 ... DIMS - 1] = 1 },
};
```


##

In [ ]:
%%bash

bart seq -h

In [ ]:
%%bash

bart seq adc grad moments

In [ ]:
%%bash

BART_DEBUG_LEVEL=5 bart seq adc grad moments
bart show -m grad

In [ ]:
plot_grad_from_cfl('grad')


In [ ]:
plot_mom_from_cfl('moments')


## Looping dimensions

BART utilizes a variety of dimensions (long[16]) for different interpretations. In general these dimensions are consistent with other concepts, e.g. the trajectory.

The current position of the loop counter is also used for setting the MDH header to be able to distinguish lines / partitions / slices / echoes  ...

| Index | Dimension Name | Description |
|-------|----------------|-------------|
| 0     | READ_DIM	| spatial position [x, y, z] for gradients/moments/trajectory |
| 1     | PHS1_DIM	| Readout dimension / Time of gradient/moment shape |
| 2     | PHS2_DIM	| First phase-encoding dimension |
| ...   | ...		| ... |
| 6     | COEFF_DIM	| = 3 (incl. pre- and post-imaging blocks, e.g. inversion pulse, delay times) |
| 7     | COEFF2_DIM	| multiple pre- (or post-) blocks |
| ...   | ...		| ... |
| 10    | TIME_DIM	| Dimension for time series |
| ...   | ...		| ... |
| 13	| SLICE_DIM	| Second phase-encoding dimension / Slices |


### Multiple radial spokes

Now we want to set up a sequence with multiple radial spokes. 

In the case of RAGA (Rational Approximation of Golden Angle) we also have to set the total number of spokes (if more than 1 full frame)


In [ ]:
%%bash

BART_DEBUG_LEVEL=5 bart seq -r4 adc grad moments


### What about the looping order?

Looping over different dimensions also allows us to flexibly change the order.

From the *seq* tool we can do this by setting the order which is basically a permutation of the dimension.

In [ ]:
%%bash

BART_DEBUG_LEVEL=4 bart seq -r2 -m2 adc grad moments


In [ ]:
%%bash
BART_DEBUG_LEVEL=4 bart seq -r2 -m2 --sequential-multislice adc grad moments


## How to obtain a trajectory from the sequence

For this purpose we can define another optional output of the *seq* tool, which describes the ADC or actual sampling.

When we check the dimensions, we see dim[READ_DIM] = 5: additionally to the trajectory, we also save the sampling time and the phase of each sample (where we can simulate FoV shifts).


In [ ]:
%%bash
bart seq -r377 adc grad moments

bart show -m adc

bart vec 1 1 0 a # set z component to zero
bart extract 0 0 3 adc - | bart fmac a - seq_traj_lin


bart traj -x 512 -y 377 -r -D trj_ref
bart nrmse trj_ref seq_traj_lin


### RAGA ordering scheme vs. chronologic ordering

Let us compare the difference between the two different RAGA ordering schemes: saving in equidistant ordering scheme (default setting) and chronologic ordering

In [ ]:
%%bash

bart seq -r377 --chrono adc_chrono grad_chrono moments_chrono

bart show -m adc_chrono

bart vec 1 1 0 a # set z component to zero
bart extract 0 0 3 adc_chrono - | bart fmac a - seq_traj_chrono


bart traj -x 256 -o 2. -y 377 -r -A -s 1 --double-base trj_ref_ga
bart nrmse  trj_ref_ga seq_traj_chrono


In [ ]:
plot_grad_from_cfl('grad')


If we plot the first two gradients we see only a small incremenent of the projection angle

In [ ]:
%%bash 

bart slice 2 0 grad grad_sp1
bart slice 2 1 grad grad_sp2


In [ ]:
plot_grad_from_cfl('grad_sp1')
plot_grad_from_cfl('grad_sp2')


In chronological ordering, e.g. the sequence which is played out, we have a way larger increment of the projection angle which is approximately the Golden Angle

In [ ]:
%%bash 

bart slice 2 0 grad_chrono grad_sp1
bart slice 2 1 grad_chrono grad_sp2

In [ ]:
plot_grad_from_cfl('grad_sp1')
plot_grad_from_cfl('grad_sp2')


### FoV shifts

We can also simulate (and validate) FoV shifts of our sequence by checking the phase of each ADC sample.

In [ ]:
%%bash

bart seq -r 377 --raga --chrono --no-spoiling adc grad mom

bart vec 1 1 0 a # set z component to zero
bart extract 0 0 3 adc - | bart fmac a - traj

bart phantom -k -t traj ksp

bart nufft -i -t traj ksp img


In [ ]:
bart_show("img")

In [ ]:
%%bash

bart seq -s 0.0256:0.0128:0 -r 377 --raga --chrono --no-spoiling adc grad mom

bart vec 1 1 0 a # set z component to zero
bart extract 0 0 3 adc - | bart fmac a - traj

bart phantom -k -t traj ksp

bart nufft -i -t traj ksp img2


In [ ]:
bart_show("img2")


In [ ]:
%%bash

bart seq -s 0.0256:0.0128:0 -r 377 --raga --no-spoiling adc grad mom

bart vec 1 1 0 a # set z component to zero
bart extract 0 0 3 adc - | bart fmac a - traj


bart slice 0 4 adc adc_phase

bart phantom -k -t traj ksp

bart fmac adc_phase ksp ksp_off

bart nufft -i -t traj ksp img
bart nufft -i -t traj ksp_off img_off

In [ ]:
bart_show("img_off", vmax=0.005)


## Pulseq export


We have already a sequence - let's export it to the open-source pulseq format

In [ ]:
%%bash

BART_DEBUG_LEVEL=5 bart seq -r1 adc grad moments single_spoke.seq

### adding an inversion pulse

In [ ]:
%%bash

BART_DEBUG_LEVEL=5 bart seq -r1 --IR_NON adc grad moments single_spoke_ir.seq

## IR - FLASH


For pulseq filese, the chronologic order is saved in each case since the interpreter runs sequentially the blocks.

In [ ]:
%%bash

bart seq --IR_NON --raga -r377          adc1 grad1 mom1 ibi_raga_377.seq
bart seq --IR_NON --raga -r377 --chrono adc2 grad2 mom2 ibi_raga_377_chrono.seq


bart nrmse adc1 adc2

# diff ibi_raga_377.seq ibi_raga_377_chrono.seq

We are only acquiring ~1.2 seconds which is too short for valid T1 mapping. 

Therefore we can increase the total number of spokes to 3 * 377 = 1131 to measure continously for about 3.5 seconds

In [ ]:
%%bash

bart seq --IR_NON --raga -t 1131 -r377 --chrono adc grad mom ibi_raga_3x377.seq

From practical perspective:

Usually adjustement scans are performed prior to the actual measurement.
In order to avoid starting from non-equilibrium magnetization state, we delay the inversion pulse initially with 10 seconds.


In [ ]:
%%bash

bart seq --IR_NON --raga -t 1131 -r377 --chrono --init_delay 10 adc grad mom ibi_raga_3x377.seq